### Pip install

In [1]:
#!pip install pennylane --upgrade

## Imports

In [2]:
import numpy
import random
import timeit

import pennylane as qml
from pennylane import numpy as np

from pennylane.optimize import AdamOptimizer

import torch
from torchvision import datasets, transforms

C:\Users\tomut\anaconda3\envs\qhack2022\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\tomut\anaconda3\envs\qhack2022\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


In [3]:
from qencode.initialize import setAB_angle, setAux, setEnt
from qencode.encoders import e1_classic
from qencode.training_circuits import swap_t
from qencode.qubits_arrangement import QubitsArrangement

## Get Data

In [4]:
def get_dataset(img_width, img_height, train):
    trainset = datasets.MNIST(root='./dataset', train=train, download=True,
                              transform=transforms.Compose(
                                  [transforms.Resize((img_width, img_height)), transforms.ToTensor()])
                              )
    return trainset

In [5]:
nr_qubits = 4
input_data = get_dataset(img_width=2, img_height=int(nr_qubits / 2), train=True)
input_data[0]

(tensor([[[0.1490, 0.1922],
          [0.1804, 0.1608]]]),
 5)

## Training node


In [6]:
shots = 1000
spec = QubitsArrangement(nr_trash=1, nr_latent=3, nr_swap=1, nr_ent=0)
print("Qubtis:", spec.qubits)

#set up the device 
dev = qml.device("default.qubit", wires=len(spec.qubits))

Qubtis: [0, 1, 2, 3, 4, 5]


In [7]:

@qml.qnode(dev)
def training_circuit_example(init_params, encoder_params, reinit_state):
    #initilaization
    setAB_angle(spec, init_params, 'X')

    setAux(spec, reinit_state)

    setEnt(spec, inputs=[1 / np.sqrt(2), 0, 0, 1 / np.sqrt(2)])

    #encoder

    for params in encoder_params:
        e1_classic(params, [*spec.latent_qubits, *spec.trash_qubits])

    #swap test 
    swap_t(spec)

    return [qml.probs(i) for i in spec.swap_qubits]


## Training parameters 

In [8]:
epochs = 50
learning_rate = 0.0003
batch_size = 8
num_samples = 64

beta1 = 0.9
beta2 = 0.999
opt = AdamOptimizer(learning_rate, beta1=beta1, beta2=beta2)

In [9]:
def fid_func(output):
    # Implemented as the Fidelity Loss
    # output[0] because we take the probability that the state after the 
    # SWAP test is ket(0), like the reference state
    fidelity_loss = 1 / output[0]
    return fidelity_loss

In [10]:
def cost(encoder_params, X):
    reinit_state = [0 for i in range(2 ** len(spec.aux_qubits))]
    reinit_state[0] = 1.0
    loss = 0.0
    for x in X:
        output = training_circuit_example(init_params=x[0], encoder_params=encoder_params, reinit_state=reinit_state)[0]
        f = fid_func(output)
        loss = loss + f
    return loss / len(X)

In [11]:
def fidelity(encoder_params, X):
    reinit_state = [0 for i in range(2 ** len(spec.aux_qubits))]
    reinit_state[0] = 1.0
    loss = 0.0
    for x in X:
        output = training_circuit_example(init_params=x[0], encoder_params=encoder_params, reinit_state=reinit_state)[0]
        f = output[0]
        loss = loss + f
    return loss / len(X)

In [12]:
def iterate_batches(X, batch_size):
    X = [torch.reshape(x[0], (1, len(spec.latent_qubits) + len(spec.trash_qubits))) for x in X]
    random.shuffle(X)

    batch_list = []
    batch = []
    for x in X:
        if len(batch) < batch_size:
            batch.append(x)

        else:
            batch_list.append(batch)
            batch = []
    if len(batch) != 0:
        batch_list.append(batch)
    return batch_list

In [13]:
training_data = [input_data[i] for i in range(num_samples)]

In [15]:
# initialize random encoder parameters
nr_encod_qubits = len(spec.trash_qubits) + len(spec.latent_qubits)
nr_par_encoder = 2 * 3 * nr_encod_qubits + 3 * (nr_encod_qubits - 1) * nr_encod_qubits
encoder_params = np.random.uniform(size=(1, nr_par_encoder), requires_grad=True)

X = [torch.reshape(x[0], (1, len(spec.latent_qubits) + len(spec.trash_qubits))) for x in training_data]

for epoch in range(epochs):
    batches = iterate_batches(X=training_data, batch_size=batch_size)
    for xbatch in batches:
        encoder_params = opt.step(cost, encoder_params, X=xbatch)

    loss_training = cost(encoder_params, X)
    fidel = fidelity(encoder_params, X)

    print("Epoch:{} |Loss:{}| Fidel:{}".format(epoch, loss_training, fidel))

Epoch:0 |Loss:1.2020792532429878|Fidel:0.8320336562772458
Epoch:1 |Loss:1.1991665110449723|Fidel:0.8340518101880596
Epoch:2 |Loss:1.1960499765901722|Fidel:0.836222241110299
Epoch:3 |Loss:1.19288759664406|Fidel:0.8384362041539796
Epoch:4 |Loss:1.1897527610877021|Fidel:0.8406424761957519
Epoch:5 |Loss:1.1866830977763272|Fidel:0.8428141779303867
Epoch:6 |Loss:1.1836990935508558|Fidel:0.8449361134535786
Epoch:7 |Loss:1.180778280786421|Fidel:0.8470234823793554
Epoch:8 |Loss:1.1779251570264273|Fidel:0.8490724649551601
Epoch:9 |Loss:1.175126329036028|Fidel:0.8510920920681266
Epoch:10 |Loss:1.1723665811686035|Fidel:0.8530929041695156
Epoch:11 |Loss:1.169662026993692|Fidel:0.8550628754629558
Epoch:12 |Loss:1.1670120386755898|Fidel:0.8570019828160237
Epoch:13 |Loss:1.1643908055593597|Fidel:0.8589286823092164
Epoch:14 |Loss:1.1618228536874893|Fidel:0.8608246783810087
Epoch:15 |Loss:1.1592880794231748|Fidel:0.8627044055002483
Epoch:16 |Loss:1.156782044589519|Fidel:0.8645708991615184
Epoch:17 |Loss